# 第6课：随机近似与随机梯度下降

> 关键词：Robbins-Monro、Stochastic Approximation、SGD、收敛性、梯度优化

---

## Part 1：通过例子介绍 Iterative Mean Estimation（均值迭代估计）

我们考虑一个基础问题：

> 如何从一个**未知分布中抽样**，估计其数学期望 $\mu = \mathbb{E}[X]$？

---

### 方法1：批量平均

若已获得 $n$ 个样本 $x_1, \dots, x_n$，用

$$
\hat{\mu}_n = \frac{1}{n} \sum_{i=1}^n x_i
$$

这就是普通的**样本均值**。

---

### 方法2：迭代均值更新

如果数据是**流式（online）**到达，能否用**迭代方式更新估计值**？

假设已有估计 $\hat{\mu}_n$，新来一个样本 $x_{n+1}$，则更新公式为：

$$
\hat{\mu}_{n+1} = \hat{\mu}_n + \alpha_n \left( x_{n+1} - \hat{\mu}_n \right)
$$

其中 $\alpha_n = \frac{1}{n+1}$，称为**学习率（step-size）**

这个思想，就是 **Robbins-Monro 随机近似算法**的雏形。

---

## Part 2：Robbins-Monro 算法介绍与例子

### 问题定义（根的随机估计）：

假设我们想解一个 **期望等式**：

$$
\mathbb{E}[F(\theta)] = 0
$$

但我们无法直接计算 $\mathbb{E}$，只能观察带噪声样本 $F_n(\theta)$，即：

$$
F_n(\theta) = F(\theta) + \text{noise}
$$

---

### Robbins-Monro 迭代算法：

$$
\theta_{n+1} = \theta_n - \alpha_n F_n(\theta_n)
$$

* $\alpha_n$：学习率序列，一般为 $\alpha_n = \frac{1}{n}$ 或 $\frac{1}{n^k}, \; 0.5 < k \leq 1$

---

### 例子：迭代估计均值

设 $F(\theta) = \theta - x$，我们希望找到 $\theta$ 使 $\mathbb{E}[\theta - X] = 0$，即 $\theta = \mathbb{E}[X]$

算法为：

$$
\theta_{n+1} = \theta_n - \alpha_n (\theta_n - x_n) = (1 - \alpha_n)\theta_n + \alpha_n x_n
$$

这与 Part 1 的均值迭代完全一致。

---

## Part 3：Robbins-Monro 收敛性与应用

### 收敛条件（Robbins-Monro 定理）：

若：

* $\sum_n \alpha_n = \infty$
* $\sum_n \alpha_n^2 < \infty$

则算法几乎必然收敛到真正的解（以概率1）。

---

### 常用步长设置：

* $\alpha_n = \frac{1}{n}$：满足条件，收敛但较慢
* $\alpha_n = \alpha$（固定步长）：不会完全收敛，但可以近似收敛（适用于非平稳环境）

---

### 应用场景：

| 应用               | 场景说明              |
| ---------------- | ----------------- |
| TD(0)、Q-learning | 用估计值替代真值，进行增量式更新  |
| 策略梯度法            | 用采样估计梯度，再用 SGD 更新 |
| DQN              | 用目标网络估值，再更新当前网络参数 |

---

## Part 4：随机梯度下降算法介绍（SGD）

在监督学习或强化学习中，我们常面对优化目标：

$$
\min_\theta \mathbb{E}_{x \sim \mathcal{D}} \left[ \ell(f_\theta(x), y) \right]
$$

但由于无法访问整个分布，**只能用样本进行梯度估计**。

---

### SGD 更新公式：

$$
\theta_{t+1} = \theta_t - \alpha_t \nabla_\theta \ell(f_\theta(x_t), y_t)
$$

每步使用一个样本或小批量进行更新，成本低、速度快、易在线学习。

---

## Part 5：SGD 例子与收敛性

### 例子：线性回归

损失函数：

$$
\ell(\theta) = \frac{1}{2} (\theta^T x - y)^2
$$

随机样本更新：

$$
\theta \leftarrow \theta - \alpha_t (\theta^T x - y) x
$$

---

### 收敛性说明：

若满足：

* 学习率满足 Robbins-Monro 条件
* 梯度方差有限

则 SGD 在凸目标函数下会收敛到全局最优。

---

### 在深度强化学习中的意义：

* DQN、Actor-Critic 中使用 SGD 更新参数化的值函数或策略
* TD 学习使用类似于 SGD 的形式更新值函数

---

## Part 6：SGD 的有趣性质

| 性质          | 说明                                |
| ----------- | --------------------------------- |
| 噪声促进跳出局部极小  | 对非凸问题（如神经网络），SGD 的随机性有助于跳出鞍点或局部极小 |
| 在线性问题中逼近最优解 | 收敛速度与特征协方差矩阵有关（见RMSProp、Adam改进）   |
| 可结合动量、正则化   | 可提升收敛速度和泛化能力                      |

---

## Part 7：SGD vs BGD vs MBGD 比较

| 方法                  | 更新单位       | 速度 | 稳定性 | 内存 | 实时性    |
| ------------------- | ---------- | -- | --- | -- | ------ |
| BGD（Batch GD）       | 所有样本       | 慢  | 稳定  | 高  | 否      |
| MBGD（Mini-Batch GD） | 一小批样本（如32） | 快  | 较稳定 | 适中 | 一般     |
| SGD                 | 单个样本       | 极快 | 不稳定 | 最小 | 支持在线学习 |

---

### 总结推荐：

| 场景            | 推荐方法     |
| ------------- | -------- |
| 模型已知，数据规模小    | BGD      |
| 模型未知，数据中等     | MBGD     |
| 流式数据、大规模、强化学习 | SGD（或变种） |

---

## 课后总结

* **Robbins-Monro** 奠定了在线优化的基础，是 TD、策略优化的理论基石
* **SGD** 是现代深度强化学习的优化核心
* **收敛性**依赖于步长设置和梯度噪声控制
* **在强化学习中，SGD 与策略值函数更新深度融合**

